In [2]:
%load_ext autoreload 
%autoreload 2

from preprocessing import FileIO, Vectorizor
from opensearch_interface import OpenSearchClient
from index_templates import youtube_body
from opensearchpy.helpers import parallel_bulk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/anaconda/envs/openai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
osclient = OpenSearchClient()

In [4]:
osclient.show_indexes()

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   semantic-538-testrun         DjBPg6CdQwKbOGhJrI4YIQ   3   1        284            0      2.9mb          2.9mb
yellow open   kw-impact-theory             2MjMun4bQYOoeUpv5UsJxg   3   1      33164            0     29.4mb         29.4mb
yellow open   security-auditlog-2023.10.10 rg-NByyNTvW0jEQQdYWdKA   1   1       1709            0      3.3mb          3.3mb
yellow open   test-kw-index                6EF4Q2xDT9Gz1wua5a2IpQ   3   1        158            0      5.6mb          5.6mb
yellow open   kw-full                      uNhdaqbnRVuyJci_L1Om8Q   3   1       6678            0     12.1mb         12.1mb
yellow open   security-auditlog-2023.09.13 nL3pRS4ATEqIiNOxME2lyw   1   1          5            0     86.4kb         86.4kb
yellow open   semantic-full                ydvbifM0Rcu0DBz-rjCHzQ   3   1       6678            0     74.2mb         74.2mb
yellow o

In [5]:
data_path = '/home/elastic/notebooks/vector_search_applications/data/impact_theory_GTE.parquet'

In [6]:
data = FileIO().load_parquet(data_path)

Shape of data: (42863, 15)
Memory Usage: 4.62+ MB


In [7]:
sorted(list(data[0].keys()))

['age_restricted',
 'author',
 'channel_id',
 'content',
 'content_embedding',
 'description',
 'episode_num',
 'keywords',
 'length',
 'playlist_id',
 'publish_date',
 'thumbnail_url',
 'title',
 'video_id',
 'views']

### KW Indexing

In [38]:
index_name = "kw-impact-theory"
youtube_body

{'settings': {'number_of_shards': 3,
  'refresh_interval': '30s',
  'index': {'knn': True}},
 'mappings': {'properties': {'title': {'type': 'text', 'index': 'true'},
   'unique_id': {'type': 'keyword', 'index': 'false'},
   'group_id': {'type': 'short', 'index': 'false'},
   'video_id': {'type': 'keyword', 'index': 'false'},
   'playlist_id': {'type': 'keyword', 'index': 'false'},
   'episode_url': {'type': 'keyword', 'index': 'false'},
   'episode_num': {'type': 'short', 'index': 'false'},
   'description': {'type': 'text', 'index': 'true'},
   'length': {'type': 'long', 'index': 'false'},
   'publish_date': {'type': 'keyword', 'index': 'false'},
   'views': {'type': 'long', 'index': 'false'},
   'thumbnail_url': {'type': 'keyword', 'index': 'false'},
   'content': {'type': 'text', 'index': 'true'},
   'content_embedding': {'type': 'knn_vector', 'dimension': 384}}}}

In [40]:
osclient.document_indexer(index_name=index_name, data=data, body_template=youtube_body)

2023-09-13 20:38:10.661 | INFO     | opensearch_interface:document_indexer:207 - The ** kw-impact-theory ** index was created
2023-09-13 20:38:10.662 | INFO     | opensearch_interface:document_indexer:218 - The # of documents to be indexed = 33164


In [44]:
osclient.indices.delete("security-audit*")

{'acknowledged': True}

In [41]:
osclient.indices.refresh(index=index_name)

{'_shards': {'total': 6, 'successful': 3, 'failed': 0}}

In [19]:
osclient.show_indexes()

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   semantic-538-testrun         DjBPg6CdQwKbOGhJrI4YIQ   3   1        284            0      2.9mb          2.9mb
yellow open   kw-impact-theory             2MjMun4bQYOoeUpv5UsJxg   3   1      33164            0     29.4mb         29.4mb
yellow open   test-kw-index                6EF4Q2xDT9Gz1wua5a2IpQ   3   1        158            0      5.6mb          5.6mb
yellow open   kw-full                      uNhdaqbnRVuyJci_L1Om8Q   3   1       6678            0     12.1mb         12.1mb
yellow open   security-auditlog-2023.09.13 nL3pRS4ATEqIiNOxME2lyw   1   1          5            0     86.4kb         86.4kb
yellow open   semantic-full                ydvbifM0Rcu0DBz-rjCHzQ   3   1       6678            0     74.2mb         74.2mb
green  open   .opendistro_security         _QeSqO4CQN2IU8VpE9hnPw   1   0         10            0     75.6kb         75.6kb
yellow o

In [18]:
osclient.cat.count(index=index_name, format="json")

[{'epoch': '1694636432', 'timestamp': '20:20:32', 'count': '33164'}]

In [19]:
def keyword_search(query: str, index: str, size: int=10, return_raw: bool=False):
    body = {
            "_source": ['content','group_id','show_link','video_id','length','publish_date','thumbnail_url','title','views'], 
            "size": size,
            "query": {
                "bool": {
                    "must": {
                        "match": {"content": query,}
                            },
                        "filter": {"bool": {"must_not": {"match_phrase": {"content": "Vishal"}}}},
                    },
                },            
            }
    response = osclient.search(body=body, index=index)
    if return_raw: 
        return response 
    else: return response['hits']['hits']

In [21]:
query = "Does trump have support in iowa"

In [23]:
# osclient.keyword_search(query, index=index_name)

In [29]:
osclient.indices.get_mapping(index=index_name)

{'kw-impact-theory': {'mappings': {'properties': {'age_restricted': {'type': 'boolean'},
    'author': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'channel_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'content': {'type': 'text'},
    'description': {'type': 'text'},
    'episode_num': {'type': 'short', 'index': False},
    'episode_url': {'type': 'keyword', 'index': False},
    'group_id': {'type': 'short', 'index': False},
    'length': {'type': 'long', 'index': False},
    'playlist_id': {'type': 'keyword', 'index': False},
    'publish_date': {'type': 'keyword', 'index': False},
    'thumbnail_url': {'type': 'keyword', 'index': False},
    'title': {'type': 'text'},
    'unique_id': {'type': 'keyword', 'index': False},
    'video_id': {'type': 'keyword', 'index': False},
    'views': {'type': 'long', 'index': False}}}}}

### Semantic Indexing

In [14]:
sem_index = 'semantic-impact-theory-gte'
model_path = '/home/elastic/notebooks/vector_search_applications/models/gte-base/' 
model = Vectorizor(model_name_or_path=model_path).model

In [20]:
osclient.indices.delete(sem_index)

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [semantic-impact-theory-gte]')

In [21]:
%%time
osclient.document_indexer(index_name=sem_index, data=data, chunk_size=1000, body_template=youtube_body, semantic_index=True)

2023-10-10 16:34:22.483 | INFO     | opensearch_interface:document_indexer:209 - The ** semantic-impact-theory-gte ** index was created
2023-10-10 16:34:22.484 | INFO     | opensearch_interface:document_indexer:216 - The # of documents to be indexed = 42863
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42863/42863 [00:26<00:00, 1590.54Docs Indexed/s]

CPU times: user 25.9 s, sys: 1.58 s, total: 27.5 s
Wall time: 27.5 s


In [20]:
def vector_search(query: str, 
                  index: str, 
                  model: Union[str, SentenceTransformer], 
                  size: int=10,
                  k: int=10,
                  return_raw: bool=False
                  ) -> Dict[str,str]:
    if isinstance(model, SentenceTransformer):
        query_embedding = model.encode(query).tolist()

    body={  "_source": ['title', 'episode_id', 'group_id', 'episode_num', 'episode_url', 'mp3_url', 'content'],
            "size": size,
            "query": 
               {"knn": {"content_embedding": {"vector": query_embedding, "k": k}}},
         }
    response = osclient.search(body=body, index=index)
    if return_raw: 
        return response 
    else: return response['hits']['hits']

In [21]:
query = 'who is more powerful, Musk or Bezos?'

In [25]:
response = osclient.vector_search(query=query, index=sem_index, model=model)
response

[{'_index': 'semantic-impact-theory-gte',
  '_id': 'W7d9FosBbbW07Kw5iXpu',
  '_score': 0.79753447,
  '_source': {'length': 3526,
   'episode_num': 24,
   'title': 'Peter Diamandis on Why A.I. Will Save the World | Impact Theory',
   'thumbnail_url': 'https://i.ytimg.com/vi/f6qdAwENFRk/hq720.jpg',
   'publish_date': '06-06-2017',
   'content': "I'm building Amazon in order to make the money to go and open the space frontier. So it was about two months ago, he sold a billion dollars of Amazon stock to continue fueling his Blue Origin space company. And then Elon Musk, who I met now back in 2001, has been as passionate about opening up space and really SpaceX is just light years ahead of most all the other aerospace companies. So you got two incredible, wealthy, passionate, driven entrepreneurs opening up space. My part of that is with a company called Planetary Resources that's going out to these asteroids that are rich in fuels, in particular hydrogen and oxygen, which is rocket fuel fr

### Reranking + Hybrid Search

In [24]:
reranker = ReRanker()

In [25]:
hybrid = osclient.hybrid_search(query, kw_index=index_name, vec_index=sem_index, model=model)
final = reranker.rerank(hybrid, query, top_k=20)

2023-09-13 20:22:28.903 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: zm0QVutAkYg-112 on index semantic-impact-theory
2023-09-13 20:22:28.904 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: SaIkelDUDic-15 on index semantic-impact-theory
2023-09-13 20:22:28.905 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: sl3XhHs6ggs-8 on index semantic-impact-theory
2023-09-13 20:22:28.906 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: OoGghm0_Q8I-30 on index semantic-impact-theory
2023-09-13 20:22:28.907 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: rKByaM5asU8-124 on index semantic-impact-theory
2023-09-13 20:22:28.908 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: kLIE93v2X-E-21 on index semantic-impact-theory
2023-09-13 20:22:28.909 | INFO     | opensearch_interface:_deduplicate_results:327 - Duplicate Hit: L7EbDo7h_To-116 o

In [71]:
def cleanup(results: List[dict]) -> List[dict]:
    final_results = []
    unique_ids = set()
    for res in results:
        unique_id = f"{res['_source']['group_id']}-{res['_source']['video_id']}"
        if unique_id in unique_ids:
            continue
        else: 
            unique_ids.add(unique_id)
            final_results.append(res)
    return final_results

In [79]:
final = cleanup(final)

In [27]:
def display_results(results: List[dict], threshhold: float=0.0, include_indexes: bool=False):
    filtered = [r for r in results if r['cross_score'] > threshhold]
    if include_indexes:
        indices = [d['_index'] for d in filtered]
        return filtered, indices
    else: return filtered

In [28]:
display_results(final, threshhold=-10, include_indexes=True)

([{'_index': 'kw-impact-theory',
   '_id': 'UNgzkIoBPTzzIQZAms8O',
   '_score': 14.798523,
   '_source': {'group_id': 12,
    'length': 5884,
    'title': 'WHY YOUR LIFE IS SO BORING... (Fix This To Find Fulfillment) | Tom Bilyeu',
    'thumbnail_url': 'https://i.ytimg.com/vi/NoSkC1hn23Q/hq720.jpg',
    'publish_date': '01-28-2023',
    'content': "Really is awesome, by the way. Even though it's black and white and it's all the shit for the TikTok generation, they're never going to go for it. But it's a really fantastic film and Orson Welles made it when he was 24 and he made it when arguably the most powerful man in the world. So imagine somebody like Bill Gates coming after you and oh God, even better, one of the guys that owns like a media conglomerate, Jeff Bezos owns Washington Post. So the guy, Rudolph Hearst, owned what at the time was the media publication. Citizen Kane is about that guy. So it would be like somebody making a movie about Jeff Bezos and that he's corrupt and lon

In [5]:
osclient.indices.get_mapping(index='kw-full')

{'kw-full': {'mappings': {'properties': {'content': {'type': 'text'},
    'episode_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'episode_num': {'type': 'short', 'index': False},
    'episode_url': {'type': 'keyword', 'index': False},
    'group_id': {'type': 'long'},
    'id': {'type': 'keyword', 'index': False},
    'mp3_url': {'type': 'keyword', 'index': False},
    'summary': {'type': 'text'},
    'title': {'type': 'text'}}}}}

In [14]:
osclient.indices.get_mapping(index='kw-538-testrun')

{'kw-538-testrun': {'mappings': {'properties': {'channel_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'content': {'type': 'text'},
    'description': {'type': 'text'},
    'episode_url': {'type': 'keyword', 'index': False},
    'group_id': {'type': 'short', 'index': False},
    'length': {'type': 'long', 'index': False},
    'playlist_id': {'type': 'keyword', 'index': False},
    'publish_date': {'type': 'keyword', 'index': False},
    'show_link': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'thumbnail_url': {'type': 'keyword', 'index': False},
    'title': {'type': 'text'},
    'video_id': {'type': 'keyword', 'index': False},
    'views': {'type': 'long', 'index': False}}}}}

In [10]:
this = hash("This is some text")

In [8]:
hash("And this is some text as well?")

872434049414064053

In [13]:
this == hash("This is some text")

True